In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
from tqdm.auto import tqdm
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
df = data.train_df('small')

In [4]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0


In [5]:
df = df.sort_values(['user_id','session_id','timestamp','step']).reset_index()

In [6]:
# find the last clickout rows
last_clickout_idxs = find_last_clickout_indices(df)
clickout_rows = df.loc[last_clickout_idxs, ['user_id','session_id','action_type','impressions']]
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|').apply(lambda x: list(map(int, x)))
clickout_rows = clickout_rows.drop('impressions', axis=1)

In [7]:
clickout_rows

,user_id,session_id,action_type,impression_list
4,0004IOZI7CKF,0146f7cb014ba,clickout item,"[2632452, 5747484, 110985, 84220, 3752262, 851..."
5,0008B0X0HC39,1b703eda68d85,clickout item,[2762974]
16,000ZN9219G41,1086064207e4f,clickout item,"[149015, 1077638, 34819, 4384770, 41629, 55250..."
20,001OCXQ9PXET,362dc2be48010,clickout item,"[2450306, 1473305, 6398796, 1161413, 13435, 23..."
21,001TEVEVUEBE,394f5ad9aa596,clickout item,"[3366614, 2670592, 6625742, 2881004, 5708270, ..."
26,002IXCPKK20W,dcdc75a82f142,clickout item,"[2730178, 2829436, 2551186, 9772462, 5092274, ..."
28,002O8H8GYKSQ,ab252cd2bafd3,clickout item,"[3469158, 3392402, 1394202, 107597, 4085894, 1..."
29,002Z7VZFWM79,d4cc248f60752,clickout item,"[56542, 79374, 3132582, 72594, 68177, 72865, 6..."
40,003728BB2ALE,eaf4b65f13e8e,clickout item,"[10875780, 8784374, 3758182, 3766642, 5058396,..."
41,003CQCD8POV7,4c2ae42150f97,clickout item,"[3139558, 2499058, 7252818, 8117350, 3994174, ..."


In [8]:
# find the interactions with numeric reference and not last clickouts
reference_rows = df[['user_id','session_id','reference','action_type','index']]
reference_rows = reference_rows[df.reference.str.isnumeric() == True].astype({'reference':'int'})
# skip last clickouts
reference_rows = reference_rows.loc[~reference_rows.index.isin(last_clickout_idxs)]
reference_rows = reference_rows.drop('action_type', axis=1)

ref_pos_serie = np.ones(reference_rows.shape[0], dtype=int) * (-1)

In [9]:
reference_rows

,user_id,session_id,reference,index
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777
17,001OCXQ9PXET,362dc2be48010,13857,884525
18,001OCXQ9PXET,362dc2be48010,13857,884526
23,002F746OAPLZ,5958b15f2af92,68760,618712
24,002F746OAPLZ,5958b15f2af92,68760,618713
25,002IXCPKK20W,dcdc75a82f142,2813356,450142
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439
32,003728BB2ALE,b55afcb9b5e96,2516686,618717


In [10]:
j = 0
clickout_indices = clickout_rows.index.values
ckidx = clickout_indices[j]
next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']
k = 0
for row in tqdm(zip(reference_rows.index, reference_rows.user_id, reference_rows.session_id, 
                    reference_rows.reference)):
    idx = row[0]
    # if the current index is over the last clickout, break
    if idx >= clickout_indices[-1]:
        break
    # find the next clickout index
    while idx > clickout_indices[j]:
        j += 1
        ckidx = clickout_indices[j]
        next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
        next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']
        next_clickout_impressions = clickout_rows.at[ckidx, 'impression_list']

    # check if row and next_clickout are in the same session
    if row[1] == next_clickout_user_id and row[2] == next_clickout_sess_id:
        try:
            ref_pos_serie[k] = next_clickout_impressions.index(row[3])
        except:
            pass
    k += 1

In [11]:
reference_rows['ref_pos'] = ref_pos_serie
reference_rows

,user_id,session_id,reference,index,ref_pos
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775,-1
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776,-1
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777,-1
17,001OCXQ9PXET,362dc2be48010,13857,884525,-1
18,001OCXQ9PXET,362dc2be48010,13857,884526,-1
23,002F746OAPLZ,5958b15f2af92,68760,618712,-1
24,002F746OAPLZ,5958b15f2af92,68760,618713,-1
25,002IXCPKK20W,dcdc75a82f142,2813356,450142,-1
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439,1
32,003728BB2ALE,b55afcb9b5e96,2516686,618717,-1


## Test

In [4]:
from extract_features.rnn.reference_position_in_last_clickout_impressions import ReferencePositionInLastClickoutImpressions
f = ReferencePositionInLastClickoutImpressions()

In [5]:
f.read_feature()

reference_position_in_last_clickout_impressions feature read


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,ref_pos
index,
8321582,-1
8321583,-1
8321584,-1
8321585,-1
8321586,-1
8321588,-1
8321589,-1
8321590,-1
8321593,-1


In [7]:
f.join_to(df)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_position_in_last_clickout_impressions feature read
reference_position_in_last_clickout_impressions onehot completed


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,ref_pos_-1
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,1
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,1
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,1
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,1
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,1
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0,1
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0,0
